In [2]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import tensorflow as tf


## Data

In [3]:

from tslearn.datasets import UCR_UEA_datasets
data_loader = UCR_UEA_datasets()

Xtrn, Ytrn, Xval, Yval = data_loader.load_dataset(
        "PenDigits")
Ytrn = np.eye(10)[Ytrn.astype(int)]
Yval = np.eye(10)[Yval.astype(int)]

In [4]:
Xtrn = np.transpose(Xtrn, (0,2,1)).reshape(-1, Xtrn.shape[1] * Xtrn.shape[2])
Xval = np.transpose(Xval, (0,2,1)).reshape(-1, Xval.shape[1] * Xval.shape[2])

In [5]:
Xtrn.shape, Ytrn.shape, Xval.shape, Yval.shape

((7494, 16), (7494, 10), (3498, 16), (3498, 10))

In [6]:
# load pt file 
import torch 
# path = '/work/users/d/d/ddinh/aaco/results/dataset_adnimlp_imputation_exclude_nan_mask_less_features.pt'
path = '/work/users/d/d/ddinh/aaco/results/dataset_digits_xgb.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [7]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [8]:
merge_action = np.expand_dims(merge_action, axis=-1)
merge_action = np.concatenate([merge_action, merge_action], axis=-1)
merge_action = np.transpose(merge_action, (0,2,1)).reshape(-1, merge_action.shape[1] * merge_action.shape[2])

In [9]:
merge_action[0]

array([1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
      dtype=float32)

In [10]:
import tensorflow as tf
# model = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp.keras')
model = XGBClassifier()
model.load_model('/work/users/d/d/ddinh/aaco/models/digits.model')

In [11]:
Xvalmasked = np.concatenate((Xval*merge_action, merge_action), 1)
val_preds = model.predict_proba(Xvalmasked) 
print(np.mean(np.round(val_preds)==Yval))

0.959748427672956


In [12]:
np.sum(merge_action, 1).mean()

4.0914807